In [ ]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=e7e7c300956984aa5668d56d4208b8728821f8770dee7622f470c98b68726275
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=5rDQiZKWsaSA2gMjnDfHqnTz8B1ui2&prompt=consent&token_usage=remote&access_type=offline&code_challenge=IThJJ8auTZeIckWcCgXb8MkF1a1KHB3QxS7tfZ7iMQE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCap6gtwwSTuHQ46AeGgYRClxfEDnuVEQrsbB-SnKGcSD6bwMP_Iu9JDv4k_NYVJQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
df = pd.read_csv("ilostat_paises_genero.csv", usecols=['ref_area.label','source.label','time','sex.label','classif2.label','obs_value'])
df['obs_value'] = df['obs_value'] * 1000
df

,ref_area.label,source.label,sex.label,classif2.label,time,obs_value
0,Afghanistan,LFS - Labour Force Survey,Sex: Male,Institutional sector: Public,2021,591751.0
1,Afghanistan,LFS - Labour Force Survey,Sex: Male,Institutional sector: Private,2021,5258677.0
2,Afghanistan,LFS - Labour Force Survey,Sex: Female,Institutional sector: Public,2021,51082.0
3,Afghanistan,LFS - Labour Force Survey,Sex: Female,Institutional sector: Private,2021,1777964.0
4,Angola,LFS - Employment Survey,Sex: Male,Institutional sector: Public,2021,716539.0
...,...,...,...,...,...,...
551,Zambia,LFS - Labour Force Survey,Sex: Female,Institutional sector: Private,2022,2532767.0
552,Zimbabwe,LFS - Labour Force Survey,Sex: Male,Institutional sector: Public,2022,256238.0
553,Zimbabwe,LFS - Labour Force Survey,Sex: Male,Institutional sector: Private,2022,1531048.0
554,Zimbabwe,LFS - Labour Force Survey,Sex: Female,Institutional sector: Public,2022,204378.0


In [ ]:
df[df['obs_value'].isna()]

,ref_area.label,source.label,sex.label,classif2.label,time,obs_value


In [ ]:
#df = df.drop(100)
df['obs_value'] = df['obs_value'].astype(int)
#df['obs_value'].isna()
df.info()
#df[df['obs_value'].isna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ref_area.label  556 non-null    object
 1   source.label    556 non-null    object
 2   sex.label       556 non-null    object
 3   classif2.label  556 non-null    object
 4   time            556 non-null    int64 
 5   obs_value       556 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 26.2+ KB


In [ ]:
df['ref_area.label'].unique()

array(['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Burundi', 'Benin',
       'Burkina Faso', 'Bangladesh', 'Bahamas', 'Bosnia and Herzegovina',
       'Belize', 'Bolivia (Plurinational State of)', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana', 'Canada', 'Switzerland',
       'Chile', "Côte d'Ivoire", 'Cameroon',
       'Congo, Democratic Republic of the', 'Cook Islands', 'Colombia',
       'Comoros', 'Cabo Verde', 'Costa Rica', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Spain', 'Ethiopia',
       'Fiji', 'France', 'Micronesia (Federated States of)',
       'United Kingdom of Great Britain and Northern Ireland', 'Georgia',
       'Ghana', 'Gambia', 'Guinea-Bissau', 'Greece', 'Grenada',
       'Guatemala', 'Guyana', 'Honduras', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Jamaica',
       'Jordan', 'Japan', 'Kenya', 'Kyrgyzstan', 'Cambodia', 'Kiribat

In [ ]:
df.rename(columns={
    'ref_area.label': 'Reference area',
    'source.label': 'Source',
    'sex.label': 'Sex',
    'classif2.label': 'Setor',
    'time': 'Time',
    'obs_value': 'Total'
}, inplace=True)
df

,Reference area,Source,Sex,Setor,Time,Total
0,Afghanistan,LFS - Labour Force Survey,Sex: Male,Institutional sector: Public,2021,591751
1,Afghanistan,LFS - Labour Force Survey,Sex: Male,Institutional sector: Private,2021,5258677
2,Afghanistan,LFS - Labour Force Survey,Sex: Female,Institutional sector: Public,2021,51082
3,Afghanistan,LFS - Labour Force Survey,Sex: Female,Institutional sector: Private,2021,1777964
4,Angola,LFS - Employment Survey,Sex: Male,Institutional sector: Public,2021,716539
...,...,...,...,...,...,...
551,Zambia,LFS - Labour Force Survey,Sex: Female,Institutional sector: Private,2022,2532767
552,Zimbabwe,LFS - Labour Force Survey,Sex: Male,Institutional sector: Public,2022,256238
553,Zimbabwe,LFS - Labour Force Survey,Sex: Male,Institutional sector: Private,2022,1531048
554,Zimbabwe,LFS - Labour Force Survey,Sex: Female,Institutional sector: Public,2022,204378


In [ ]:
df['Sex'] = df['Sex'].str.replace('Sex: ', '')
df['Setor'] = df['Setor'].str.replace('Institutional sector: ', '')
print(df)

    Reference area                     Source     Sex    Setor  Time    Total
0      Afghanistan  LFS - Labour Force Survey    Male   Public  2021   591751
1      Afghanistan  LFS - Labour Force Survey    Male  Private  2021  5258677
2      Afghanistan  LFS - Labour Force Survey  Female   Public  2021    51082
3      Afghanistan  LFS - Labour Force Survey  Female  Private  2021  1777964
4           Angola    LFS - Employment Survey    Male   Public  2021   716539
..             ...                        ...     ...      ...   ...      ...
551         Zambia  LFS - Labour Force Survey  Female  Private  2022  2532767
552       Zimbabwe  LFS - Labour Force Survey    Male   Public  2022   256238
553       Zimbabwe  LFS - Labour Force Survey    Male  Private  2022  1531048
554       Zimbabwe  LFS - Labour Force Survey  Female   Public  2022   204378
555       Zimbabwe  LFS - Labour Force Survey  Female  Private  2022  1049787

[556 rows x 6 columns]


In [ ]:
df.head()

,Reference area,Source,Sex,Setor,Time,Total
0,Afghanistan,LFS - Labour Force Survey,Male,Public,2021,591751
1,Afghanistan,LFS - Labour Force Survey,Male,Private,2021,5258677
2,Afghanistan,LFS - Labour Force Survey,Female,Public,2021,51082
3,Afghanistan,LFS - Labour Force Survey,Female,Private,2021,1777964
4,Angola,LFS - Employment Survey,Male,Public,2021,716539


In [ ]:
df = df.rename(columns={'Time':'ano_pesquisa','Reference area':'pais','Source':'fonte_pesquisa','Total':'quantidade','Sex':'genero','Setor':'setor'})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            556 non-null    object
 1   fonte_pesquisa  556 non-null    object
 2   genero          556 non-null    object
 3   setor           556 non-null    object
 4   ano_pesquisa    556 non-null    int64 
 5   quantidade      556 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 26.2+ KB


In [ ]:
min(df['ano_pesquisa'])

2013

In [ ]:
max(df['ano_pesquisa'])

2023

In [ ]:
unique_countries = df['setor'].unique()
print(unique_countries)

translations = {
    'Private': 'Privado',
    'Public': 'Público'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['setor'] = df['setor'].replace(translations)

print(df)

['Public' 'Private']
            pais             fonte_pesquisa  genero    setor  ano_pesquisa  \
0    Afghanistan  LFS - Labour Force Survey    Male  Público          2021   
1    Afghanistan  LFS - Labour Force Survey    Male  Privado          2021   
2    Afghanistan  LFS - Labour Force Survey  Female  Público          2021   
3    Afghanistan  LFS - Labour Force Survey  Female  Privado          2021   
4         Angola    LFS - Employment Survey    Male  Público          2021   
..           ...                        ...     ...      ...           ...   
551       Zambia  LFS - Labour Force Survey  Female  Privado          2022   
552     Zimbabwe  LFS - Labour Force Survey    Male  Público          2022   
553     Zimbabwe  LFS - Labour Force Survey    Male  Privado          2022   
554     Zimbabwe  LFS - Labour Force Survey  Female  Público          2022   
555     Zimbabwe  LFS - Labour Force Survey  Female  Privado          2022   

     quantidade  
0        591751  
1     

In [ ]:
unique_countries = df['genero'].unique()
print(unique_countries)

translations = {
    'Male': 'Homem',
    'Female': 'Mulher'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['genero'] = df['genero'].replace(translations)

print(df)

['Male' 'Female']
            pais             fonte_pesquisa  genero    setor  ano_pesquisa  \
0    Afghanistan  LFS - Labour Force Survey   Homem  Público          2021   
1    Afghanistan  LFS - Labour Force Survey   Homem  Privado          2021   
2    Afghanistan  LFS - Labour Force Survey  Mulher  Público          2021   
3    Afghanistan  LFS - Labour Force Survey  Mulher  Privado          2021   
4         Angola    LFS - Employment Survey   Homem  Público          2021   
..           ...                        ...     ...      ...           ...   
551       Zambia  LFS - Labour Force Survey  Mulher  Privado          2022   
552     Zimbabwe  LFS - Labour Force Survey   Homem  Público          2022   
553     Zimbabwe  LFS - Labour Force Survey   Homem  Privado          2022   
554     Zimbabwe  LFS - Labour Force Survey  Mulher  Público          2022   
555     Zimbabwe  LFS - Labour Force Survey  Mulher  Privado          2022   

     quantidade  
0        591751  
1       5

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            556 non-null    object
 1   fonte_pesquisa  556 non-null    object
 2   genero          556 non-null    object
 3   setor           556 non-null    object
 4   ano_pesquisa    556 non-null    int64 
 5   quantidade      556 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 26.2+ KB


In [ ]:
df=df.rename(columns={'País':'pais'})

In [ ]:
df['pais'].unique()

array(['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Burundi', 'Benin',
       'Burkina Faso', 'Bangladesh', 'Bahamas', 'Bosnia and Herzegovina',
       'Belize', 'Bolivia (Plurinational State of)', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana', 'Canada', 'Switzerland',
       'Chile', "Côte d'Ivoire", 'Cameroon',
       'Congo, Democratic Republic of the', 'Cook Islands', 'Colombia',
       'Comoros', 'Cabo Verde', 'Costa Rica', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Spain', 'Ethiopia',
       'Fiji', 'France', 'Micronesia (Federated States of)',
       'United Kingdom of Great Britain and Northern Ireland', 'Georgia',
       'Ghana', 'Gambia', 'Guinea-Bissau', 'Greece', 'Grenada',
       'Guatemala', 'Guyana', 'Honduras', 'Indonesia',
       'Iran (Islamic Republic of)', 'Iraq', 'Israel', 'Jamaica',
       'Jordan', 'Japan', 'Kenya', 'Kyrgyzstan', 'Cambodia', 'Kiribat

In [ ]:
# Lista de valores permitidos
lista_valores = ['South Africa', 'Australia', 'Argentina', 'Bolivia (Plurinational State of)', 'Brazil', 'Chile',
       'Colombia', 'Dinamarca', 'United States of America', 'France', 'Mexico',
       'Peru', 'Suécia', 'Uruguay', 'United Kingdom of Great Britain and Northern Ireland']

# Filtrar e manter apenas as linhas onde 'coluna_especifica' está na lista
df = df[df['pais'].isin(lista_valores)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 16 to 547
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            52 non-null     object
 1   fonte_pesquisa  52 non-null     object
 2   genero          52 non-null     object
 3   setor           52 non-null     object
 4   ano_pesquisa    52 non-null     int64 
 5   quantidade      52 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.8+ KB


In [ ]:
df['pais'].unique()

array(['Argentina', 'Australia', 'Bolivia (Plurinational State of)',
       'Brazil', 'Chile', 'Colombia', 'France',
       'United Kingdom of Great Britain and Northern Ireland', 'Mexico',
       'Peru', 'Uruguay', 'United States of America', 'South Africa'],
      dtype=object)

In [ ]:
unique_countries = df['pais'].unique()
print(unique_countries)

translations = {
    'South Africa': 'África do Sul',
    'Argentina': 'Argentina',
    'Brazil': 'Brasil',
    'Chile': 'Chile',
    'Colombia': 'Colômbia',
    'United States of America': 'Estados Unidos',
    'France': 'França',
    'Mexico': 'México',
    'Peru': 'Peru',
    'Bolivia (Plurinational State of)': 'Bolívia',
    'Uruguay': 'Uruguai',
    'France' : 'França',
    'Australia' : 'Austrália',
    'United Kingdom of Great Britain and Northern Ireland' : 'Reino Unido'
}

# Substituindo os valores da coluna 'country' pelos valores traduzidos
df['pais'] = df['pais'].replace(translations)

df['pais'].unique()

['Argentina' 'Australia' 'Bolivia (Plurinational State of)' 'Brazil'
 'Chile' 'Colombia' 'France'
 'United Kingdom of Great Britain and Northern Ireland' 'Mexico' 'Peru'
 'Uruguay' 'United States of America' 'South Africa']


array(['Argentina', 'Austrália', 'Bolívia', 'Brasil', 'Chile', 'Colômbia',
       'França', 'Reino Unido', 'México', 'Peru', 'Uruguai',
       'Estados Unidos', 'África do Sul'], dtype=object)

In [ ]:
df_brasil = df[df['pais'] == 'Brasil']
df_brasil

,pais,fonte_pesquisa,genero,setor,ano_pesquisa,quantidade
64,Brasil,HS - Continuous National Household Sample Survey,Homem,Público,2023,5142221
65,Brasil,HS - Continuous National Household Sample Survey,Homem,Privado,2023,51508263
66,Brasil,HS - Continuous National Household Sample Survey,Mulher,Público,2023,6966064
67,Brasil,HS - Continuous National Household Sample Survey,Mulher,Privado,2023,35665217


In [ ]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('genero','STRING',description='Gênero da pessoa'),
bigquery.SchemaField('quantidade','INTEGER',description='total de pessoas que trabalham'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
 bigquery.SchemaField('ano_pesquisa','INTEGER',description='Ano de coleta da informação'),

 ]

In [ ]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
dataset_ref = client.dataset('perfil_remuneracao')

In [ ]:
table_ref = dataset_ref.table('ILOSTAT_paises_selecionados_genero_v4')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=85831c54-b275-44ec-803f-de8e6acc1686>

In [ ]:
df['pais'].unique()

array(['Argentina', 'Austrália', 'Bolívia', 'Brasil', 'Chile', 'Colômbia',
       'França', 'Reino Unido', 'México', 'Peru', 'Uruguai',
       'Estados Unidos', 'África do Sul'], dtype=object)

In [ ]:
df.to_csv('ilostat_genero_latest_period_v3.csv', index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 16 to 547
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   pais            52 non-null     object
 1   fonte_pesquisa  52 non-null     object
 2   genero          52 non-null     object
 3   setor           52 non-null     object
 4   ano_pesquisa    52 non-null     int64 
 5   quantidade      52 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 2.8+ KB
